<a href="https://colab.research.google.com/github/android-kunjapppan/JanataHack-Computer-vision/blob/master/Emergency_vehicle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import io
import cv2
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import warnings

warnings.filterwarnings('ignore')

In [68]:
from google.colab import files
uploaded = files.upload()

Saving test_vc2kHdQ.csv to test_vc2kHdQ.csv


In [0]:
df= pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [6]:
df.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path='/content/drive/My Drive/images'

In [14]:
images=[]
for image_name in tqdm(df['image_names']):
  image = os.path.join(path,str(image_name))
  image = cv2.imread(image)
  image_resized = cv2.resize(image,(224,224))
  images.append(np.array(image_resized,"float32"))
  


100%|██████████| 1646/1646 [13:33<00:00,  2.02it/s]


In [0]:
labels = df.iloc[:,-1].values

In [0]:

X_train, X_test, y_train, y_test = train_test_split(images,labels, test_size=0.35,random_state=42)



In [22]:
X_train = np.array(X_train)/255.0
X_test = np.array(X_test)/255.0
y_train = np.array(y_train)
y_test = np.array(y_test)


print(X_train.shape)
print(y_train.shape)
X_test.shape, y_test.shape

(1069, 224, 224, 3)
(1069,)


((577, 224, 224, 3), (577,))

In [63]:
images=np.array(images)/255.0
labels=np.array(labels)

images = np.reshape(images,(images.shape[0],224,224,3))
labels = to_categorical(labels)

images.shape, labels.shape

((1646, 224, 224, 3), (1646, 2))

In [23]:
X_train = np.reshape(X_train,(X_train.shape[0],224,224,3))
X_test = np.reshape(X_test,(X_test.shape[0],224,224,3))

print(X_train.shape)
print(y_train.shape)
X_test.shape, y_test.shape

(1069, 224, 224, 3)
(1069,)


((577, 224, 224, 3), (577,))

In [0]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [0]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Flatten,MaxPooling2D,Dropout,GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications import MobileNetV2
from keras.models import Model


In [96]:
num_features = 64
num_labels = 2
width, height, channels = 224, 224, 3
train_aug = ImageDataGenerator(rescale = 1./255,rotation_range=20,horizontal_flip=True,zoom_range=0.2,shear_range=0.2)
test_aug = ImageDataGenerator(rescale = 1./255)


base_model = MobileNetV2(input_shape=(width,height,channels),
                                               include_top=False,
                                               weights='imagenet')

= base_model.output

base_model.trainable = False


global_average_layer = GlobalAveragePooling2D()
feature_batch_average = global_average_layer(X)


prediction_layer = Dense(2,activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)


model = Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

"""#create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(train_aug.flow(images,labels,batch_size=BATCH_SIZE),epochs=5,)"""



"""model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, channels), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))



model.add(Flatten())

model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))"""

"model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, channels), data_format='channels_last'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\nmodel.add(Dropout(0.2))\n\nmodel.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\nmodel.add(Dropout(0.3))\n\nmodel.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\nmodel.add(Dropout(0.3))\n\n\n\nmodel.add(Flatten())\n\nmodel.add(Dense(2*2*num_features, activation='relu'))\nmodel.add(Dropout(0.4))\n\nmodel.add(Dense(2*num_features, activation='relu'))\nmodel.add(Dropout(0.4))\n\nmodel.add(Dense(num_features, activation='relu'))\nmodel.add(Dropout(0.5))\n\nmodel.add(Dense(num_labels, activation='softmax'))"

In [0]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [99]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_7 ( (None, 1280)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [101]:
initial_epochs = 10
validation_steps=20
model.fit(train_aug.flow(images,labels,batch_size=BATCH_SIZE),epochs=5)


Epoch 1/5
206/206 [==============================] - 101s 492ms/step - loss: 0.7052 - accuracy: 0.5680
Epoch 2/5
206/206 [==============================] - 98s 476ms/step - loss: 0.6881 - accuracy: 0.6349
Epoch 3/5
206/206 [==============================] - 99s 481ms/step - loss: 0.6765 - accuracy: 0.6665
Epoch 4/5
206/206 [==============================] - 99s 481ms/step - loss: 0.6668 - accuracy: 0.6883
Epoch 5/5
206/206 [==============================] - 98s 475ms/step - loss: 0.6651 - accuracy: 0.6889


In [0]:
base_model.trainable = True

In [0]:
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [0]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [106]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

model.fit(train_aug.flow(images,labels,batch_size=BATCH_SIZE),epochs=total_epochs)

Epoch 1/20
206/206 [==============================] - 145s 704ms/step - loss: 0.6489 - accuracy: 0.7266
Epoch 2/20
206/206 [==============================] - 140s 680ms/step - loss: 0.6268 - accuracy: 0.7722
Epoch 3/20
206/206 [==============================] - 140s 681ms/step - loss: 0.6193 - accuracy: 0.7789
Epoch 4/20
206/206 [==============================] - 142s 688ms/step - loss: 0.6132 - accuracy: 0.7825
Epoch 5/20
206/206 [==============================] - 143s 696ms/step - loss: 0.6022 - accuracy: 0.8056
Epoch 6/20
206/206 [==============================] - 144s 697ms/step - loss: 0.5950 - accuracy: 0.8354
Epoch 7/20
206/206 [==============================] - 143s 695ms/step - loss: 0.5911 - accuracy: 0.8360
Epoch 8/20
206/206 [==============================] - 143s 694ms/step - loss: 0.5904 - accuracy: 0.8293
Epoch 9/20
206/206 [==============================] - 144s 698ms/step - loss: 0.5846 - accuracy: 0.8463
Epoch 10/20
206/206 [==============================] - 142s 689m

In [107]:
predicted1 = model.predict(test_images)
predicted1 = pd.DataFrame(predicted)
pred0 = np.round(predicted1)
pred2 =pred0.iloc[:,1:]
pred2

,1
0,0.0
1,1.0
2,0.0
3,1.0
4,0.0
...,...
701,1.0
702,0.0
703,1.0
704,0.0


In [0]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [0]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [67]:
model.fit(train_aug.flow(images,labels,batch_size=BATCH_SIZE),epochs=5)


Epoch 1/5
206/206 [==============================] - 206s 1s/step - loss: 0.5972 - accuracy: 0.7521
Epoch 2/5
206/206 [==============================] - 202s 982ms/step - loss: 0.4836 - accuracy: 0.7685
Epoch 3/5
206/206 [==============================] - 202s 979ms/step - loss: 0.4611 - accuracy: 0.7904
Epoch 4/5
206/206 [==============================] - 203s 987ms/step - loss: 0.4611 - accuracy: 0.7886
Epoch 5/5
206/206 [==============================] - 203s 983ms/step - loss: 0.4731 - accuracy: 0.7776


In [0]:
test=pd.read_csv(io.BytesIO(uploaded['test_vc2kHdQ.csv']))

In [70]:
test_images=[]
for image_name in tqdm(test['image_names']):
  image = os.path.join(path,str(image_name))
  image = cv2.imread(image)
  image_resized = cv2.resize(image,(224,224))
  test_images.append(np.array(image_resized,"float32"))

100%|██████████| 706/706 [06:04<00:00,  1.94it/s]


In [0]:
test_images=np.array(test_images)/255.0
test_images = np.reshape(test_images,(test_images.shape[0],224,224,3))
predicted = model.predict(test_images)
predicted = pd.DataFrame(predicted)

In [90]:
predicted.head()
pred = np.round(predicted)
pred1 =pred.iloc[:,1:]
pred1

,1
0,0.0
1,1.0
2,0.0
3,1.0
4,0.0
...,...
701,1.0
702,0.0
703,1.0
704,0.0


In [0]:
test_last1=pd.concat([test,pred2],axis=1)
test_last1.to_csv('final4.csv')

In [0]:
BATCH_SIZE = 8
EPOCHS = 20       

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=1)

checkpoint = ModelCheckpoint('vehicles_model.h5',monitor='val_loss',verbose=0,
                            save_best_only='True',mode='auto')

In [0]:
history = model.fit(train_aug.flow(X_train,y_train,batch_size=BATCH_SIZE),
                    callbacks=[lr_reducer,checkpoint],epochs=EPOCHS,
                    validation_data=test_aug.flow(X_test,y_test))